In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os

os.getcwd()

'/media/christian/D/Bootcamp/Alumno/ds_thebridge_6_22/EDA/linkedin_job_analysis'

In [3]:
# Import Liberies
from bs4 import BeautifulSoup as bs
import pandas as pd
import math
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime


# Selenium options
options = Options()

# Linkedin limits
elem_p_page = 25  # Linkedin shows only 25 results per page, max 1000 in total
linkedin_max_pages = 40 # max pages of linkedin results ! Linkedin even only shows 12 pages sometimes

# base_search_page: https://www.linkedin.com/jobs/search/?keywords=developer%20python&location=Germany&f_TPR=r2592000

# User Login Information, put your linkedin login email and password
user_data= "your email o linkedin username"
password = "your password"

# Search keywords and place change it
searchterm_org = '"data scientist"'
searchterms = searchterm_org.split()
searchterm = searchterms[0] + '"' + ' AND ' + '"'+ searchterms[1]
searchterm = searchterm_org
searchlocation = "Spain" # The country or location of the search

# Initialize
#periode = "r2592000"  # the code for month actually not used
number_of_pages = 0 # Don't change


# The base URL 
joburl_base = "https://www.linkedin.com/jobs/view/"


# geoID is the Linkedin country code, i.e Denmark, geoID 104514075, Ireland 104738515
geoid = {"Denmark":"104514075","Ireland":"104738515","Spain":"105646813",
"Austria":"103883259","Francia":"105015875","Italy":"103350119","Germany":"101282230",
"Netherlands":"102890719", "Belgium":"100565514", "Sweden":"105117694", "Norway":"103819153",
"Finland":"100456013","Estonia":"102974008","Czechia":"104508036","Hungary":"100288700", "Greece":"104677530",
"Malta":"100961908", "United Kingdom":"101165590", "Poland":"105072130", "Portugal":"100364837", "Iceland":"105238872"
}


# Selenium options
options.add_argument(' — headless')
options.add_argument('— no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument("--disable-notifications")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-javascript")
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
 
# Login page
URL = "https://www.linkedin.com/login?emailAddress=&amp;fromSignIn=&amp;fromSignIn=true&amp"


# Function for creating waiting
def sleep_flex(start=4,end=20):
    """ 
    Function for create waiting delays based on random in order
    to reduce bot detection on Linkedin
    
    """
    rand_start = random.randint(start,end)
    rand_end = random.randint(start+3,end+3)
    if rand_end>rand_start:
        sleep_flexo = random.randint(rand_start,rand_end)
        return time.sleep(sleep_flexo)
    else:
        return sleep_flex(5,10)
    
  
# Connecting to the first URL page and doing the sign-in

def doing_login (URL, driver,user_data,password):
    """
    Function for doing the login in Linkedin
    Args:
    URL (str) : Url for the login screen
    driver (object) : Selenium driver object
    user_data (str) : The username / email for linkedin
    password (str) : The user password for linkedin

    """
    driver.get(URL)

    user_input = driver.find_element(By.ID, 'username')
    user_input.click()
    sleep_flex(3,6)
    user_input.send_keys(user_data)
    sleep_flex(2,7)
    password_input = driver.find_element(By.ID, 'password')
    sleep_flex(2,7)
    password_input.click()
    sleep_flex(3,8)
    password_input.send_keys(password)
    sleep_flex(2,8)

    # Signin
    signin = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button').click()
    sleep_flex(2,9)
    # Got to jobs page
    job_search_main_page  = driver.find_element(By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
    return signin,job_search_main_page
    # End function


def searchforjob (driver:object,searchterm:str,searchlocation:str):
    """ Function for doing the search of the job returnin the number of pages
    Args:
    driver (object) : Selenium driver object
    searchterm (str) : String of keywords
    searchlocation (str) : String of the country or city

    Returns:
    number_of_pages (int) : Number of pages, max. 40
    max_results (int) : Number of results, max. 1000
    """

    sleep_flex(4,10)
    job_search_field= driver.find_element(By.XPATH,"//*[starts-with(@id, 'jobs-search-box-keyword-id')]")
    job_search_location = driver.find_element(By.XPATH,"//*[starts-with(@id, 'jobs-search-box-location-id')]")


    # Doing clicks and inputs in the jobsearch form
    job_search_field.click()
    sleep_flex(2,15)
    job_search_field.send_keys(searchterm)
    sleep_flex(3,8)
    job_search_location.click()
    sleep_flex(3,15)
    job_search_location.send_keys(searchlocation)
    sleep_flex(4,16)
    job_search_location.send_keys(Keys.RETURN)
    sleep_flex(1,12)

    # Doing the Soup
    html = driver.page_source
    soup = bs(html, "html.parser")

    print(soup)

    # Creating a Dataframe for saving the overall results of the search
    country_results = pd.DataFrame(columns=["country","searchterm","number_results","timestamp"])


    try:
        # Calculation of max pages, although linkedin permits showing only 1000 results = 40 pages max
        number_results = soup.find_all(class_="display-flex t-12 t-black--light t-normal jobs-search-results-list__text")
        
        # Getting overall results for saving in CSV
        for i in number_results:
          #  print(i.get_text(strip=True))
            qty_results = int(i.get_text(strip=True).replace(",","").split(" ")[0])
            number_of_pages = math.ceil(qty_results/elem_p_page)
            timestamp= datetime.now()
            to_append= [searchlocation,searchterm_org,qty_results,timestamp]
            a_series = pd.Series(to_append, index = country_results.columns)
            country_results = country_results.append(a_series, ignore_index=True)
            

            if qty_results >= 1000:
                max_results = 1000  # Linkedin only show max 1000 results
            else:
                max_results = qty_results


        print (max_results)


        if number_of_pages >= linkedin_max_pages: 
            number_of_pages = 40
        else:
            number_of_pages = number_of_pages


        ## Using of result numbers for iteration until 1000 results
        print (number_of_pages)

        ## save country results to CSV
        country_results.to_csv('jobs_country.csv', mode='a', index=False, header=False, sep="|")

    
    except:
        print ("error page_result")

    return number_of_pages,max_results
    # End of function



def perform_click  (number_of_pages:int, driver:object, joburl_base:str):
    """ Does clicking on the result pages and looks for the results found, creates a new
    url for each job in order to check details with the scraper

    Args:
    number_of_pages (int) : number of result pages (max. 40)
    driver (obj) : the selenium driver obj
    joburl_base : the base url for the job detail page

    Return:
    jobs_detail_url (list) : returns list with all the job-detail URLS
    
    """

    # create list for saving job-urls
    jobs_detail_url = []

    # When there are more than 1 result pages use the following loop else the steps in else
    if number_of_pages > 1:
        for e in range(1,number_of_pages+1):

                if e % 10 == 0:
                    time.sleep(5)
                    driver.refresh()
                    driver.maximize_window()
                    time.sleep(2)
                    driver.back()
                    time.sleep(2)
                    driver.forward()



                sleep_flex(4,15)
                get_url = driver.current_url
                print("The current url is:"+str(get_url))
                print (e)
                page = "Page "+ str(e)
                print (page)
                time.sleep(3)

                    ## Working example of clicking on page in pagination for scraping jobs
                try:
                    
                    
                        try:
                            next_page_btn = driver.find_element(By.CSS_SELECTOR,f"button[aria-label='{page}']")
                            next_page_btn.click()
                            print(next_page_btn)
                            sleep_flex(2,9)
                        except:

                            try:
                                next_page_btn = driver.find_element(By.XPATH, f"//button[contains(@aria-label,'{page}')]")
                                next_page_btn.click()
                                print (next_page_btn)
                                sleep_flex(2,9)
                            except:
                                next_page_btn = driver.find_element(By.CSS_SELECTOR,f"li[data-test-pagination-page-btn='{e}']")
                                next_page_btn.click()
                                print (next_page_btn)
                                sleep_flex(2,9)

                        
                        
                        html = driver.page_source
                        soup = bs(html, "html.parser")
                        sleep_flex(8,22)


                        ## Linkedin shows 25 job offers per page
                        print ("---"*17)
                        try:

                            selector = soup.find_all("li",attrs={"data-occludable-job-id":True})
                            for i in selector:
                            # print (i)
                                try:
                                    ### Here we add the url of the job to the list, based on the id found and the baseURL for jobview
                                    jobs_detail_url.append(joburl_base+i["data-occludable-job-id"]) 
                                # print (i)

                                except:
                                    print("error data-occludable-job-id")

                        except:
                            print ("error customized attribute")


                        print ("----"*15)
                

                except:
                        print (f"error click on {e} page")

                        while not next_page_btn:
                            sleep_flex(2,9)
                            driver.refresh()
                            #driver.maximize_window()
                            sleep_flex(2,9)
                            driver.back()
                            sleep_flex(2,9)
                            driver.forward()


                            try:
                                    next_page_btn = driver.find_element(By.CSS_SELECTOR,f"button[aria-label='{page}']")
                                    next_page_btn.click()
                                    print(next_page_btn)
                                    sleep_flex(2,5)
                            except:

                                try:
                                    next_page_btn = driver.find_element(By.XPATH, f"//button[contains(@aria-label,'{page}')]")
                                    next_page_btn.click()
                                    print (next_page_btn)
                                    sleep_flex(2,5)
                                except:
                                    next_page_btn = driver.find_element(By.CSS_SELECTOR,f"li[data-test-pagination-page-btn='{e}']")
                                    next_page_btn.click()
                                    print (next_page_btn)
                                    sleep_flex(2,5)



    else:
            # When there is only one result page do the following
            html = driver.page_source
            soup = bs(html, "html.parser")
            #print (soup)
            sleep_flex(8,22)
            try:

                selector = soup.find_all("li",attrs={"data-occludable-job-id":True})
                for i in selector:
                # print (i)
                    try:
                
                        jobs_detail_url.append(joburl_base+i["data-occludable-job-id"])
                    # print (i)

                    except:
                        print("error data-occludable-job-id")

            except:
                print ("error customized attribute")



    return  jobs_detail_url    
    # End function                    



def scrap_page (lista_urls:list, searchterm_org:str, searchlocation:str, driver):
    """ Function for scraping a Linkedin Job offer page from a list of linkedin-job urls.
    returns a list of the pages scraped and appends a CSV file

    Args:
    lista_urls (list) : List of urls for which the scraping should be done
    searchterm_org (str) : String of the original searchterm which is used as information in the csv
    searchlocation (str) : The place or country for which the search was performed
    
    Return:
    list_jobs (pd object) : List of the jobs scraped
    """

    # Create empty Df
    lista_jobs = pd.DataFrame( columns = [
        'id',"country", 'searchterm', 'job_title', 'company','job_location',
        'time','applicants','workplace','working_time','job_level','company_size','industry','job_details','job_url','timestamp'
    ])
    

    # File for saving
    searchterm = searchterm_org.replace('"',"")
    f = f'linkedin_{searchterm}_{searchlocation}.csv'
    

    if os.path.exists(f): 
        lista_jobs.to_csv(f, mode='a', index=False, header=True, sep="|")
    else:
        lista_jobs.to_csv(f, mode='w', index=False, header=True, sep="|")

    # Loop through the url list of jobs
    for i,e in enumerate(lista_urls):

        url = e
        sleep_flex(2,8)
   
        try:
            driver.get(url)
            content = driver.page_source
            soup1 = bs(content, "html5lib")

           ## Get actual URL
           # get_url = driver.current_url
           # print("The current url is:"+str(get_url))
            sleep_flex(1,5)


            ## Reading the soup of the elements found and prepare for DF and CSV, in cases where soup doesn't work we use Selenium
            
            try:
               job_title = soup1.find(class_="t-24 t-bold jobs-unified-top-card__job-title")
               title = job_title.get_text(strip=True)
               print(title)

            except:
                print("error reading job_title")
                title = " "

            try:
                job_company= soup1.find(class_="jobs-unified-top-card__company-name")
                company = job_company.get_text(strip=True)
                print (company)
                
            except:
                print("error reading job company" )
                company = " "
            
            try:
                job_location= soup1.find(class_="jobs-unified-top-card__bullet")
                location = job_location.get_text(strip=True)
                print (location)
            except:
                print("error reading job location")
                location = " "

            try:
                job_workplace = soup1.find(class_="jobs-unified-top-card__workplace-type")
                workplace = job_workplace.get_text(strip=True)
                print (workplace)

            except:
                print("error reading workplace")
                workplace = " "
            

            try:

                content = driver.find_element(By.CLASS_NAME, "jobs-unified-top-card__applicant-count")
                #print ("content",content)
                job_applicants = content.text
                applicants = job_applicants
                print (applicants)
                
                

            except:
                print("error reading applicants")
                applicants = " "
                

            try:
                content = driver.find_element(By.XPATH, "//li[@class='jobs-unified-top-card__job-insight'][1]/span[1]").text
                job_caracteristic = content.split("·",1)
                working_time = job_caracteristic[0].strip()
                job_level = job_caracteristic[1].strip()
                print (working_time+","+job_level)
            
            except:
                print ("error reading working time and job level")



            try:
                content = driver.find_element(By.XPATH, "//li[@class='jobs-unified-top-card__job-insight'][2]/span[1]").text
                industry_caracteristics = content.split("·",1)
                company_size = industry_caracteristics[0].strip()
                industry = industry_caracteristics[1].strip()
                print(company_size+","+industry)

            except:
                print ("error reading company size and industry")



            try:
                job_time= soup1.find(class_="jobs-unified-top-card__posted-date")
                published = job_time.get_text(strip=True)
                print(published)

            except:
                print ("error reading job-offer posting date")
                published = " "


            # Log
            print ("---"*16)
            print(job_title,job_company,job_location,job_workplace,job_time)
            print ("---"*16)


            try:
                job_details = soup1.find(id="job-details").get_text()
              #  print(job_details)
            
            except:
                job_details2 = soup1.find_all('div',class_="mt5 jobs-box__html-content t-14")
                print("error reading details")

                for i,e in enumerate(job_details2):
                    job_details = e.get_text(strip=True)
                   # print(job_details)  

            

            #### new row creation - and appending CSV
            timestamp= datetime.now()
            new_job = {"id":i,"country":searchlocation,"searchterm":searchterm_org.replace('"',''),"job_title":title,"company":company,
            "job_location":location,"time":published,"applicants":applicants,"workplace":workplace,
            'working_time':working_time,'job_level':job_level,'company_size':company_size,'industry':industry,
             "job_details":job_details,"job_url":url, "timestamp":timestamp}
            
            row = pd.DataFrame(new_job, index=[0])
            data = [lista_jobs, row]
            lista_jobs = pd.concat(data, ignore_index=True, sort=False)
            #lista_jobs = lista_jobs.append(new_job, ignore_index=True)

            # Writing to CSV, separator |, you can use other but not point, comma etc
            row.to_csv(f, mode='a', index=False, header=False, sep="|")
            
            

        except:
            print("error: one of the information could not be scrapped, check the class names")


    return lista_jobs
    # End of function 


# Login
doing_login (URL, driver,user_data,password)
# Search for job
number_of_pages,max_results = searchforjob (driver,searchterm,searchlocation)
# Scraping 
print ("----"*15)
## Clicking through pages for getting the job_detail urls                       
jobs_detail_url  = perform_click(number_of_pages, driver,joburl_base)
jobs_db = scrap_page(jobs_detail_url,searchterm_org,searchlocation, driver)


# Delay and close Selenium Driver and browser
time.sleep(50)
driver.close()
print("End of scraping")





<html class="theme theme--mercado artdeco" lang="en"><head>
<script async="" charset="utf-8" id="utag_145" src="https://platform.linkedin.com/litms/vendor/google/gtag-cm-dv360-sa360.js?id=DC-9261636" type="text/javascript"></script><script async="" charset="utf-8" id="utag_63" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript"></script><script async="" charset="utf-8" id="utag_128" src="https://platform.linkedin.com/litms/vendor/google/gtag-adwords.js?id=AW-867846157" type="text/javascript"></script><script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>
<title>(7) "data scientist" Jobs in Germany | LinkedIn</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta class="mercado-icons-sprite" content="

/tmp/ipykernel_5698/2980748130.py:174: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_results = country_results.append(a_series, ignore_index=True)


The current url is:https://www.linkedin.com/jobs/search/?currentJobId=3283954806&geoId=101282230&keywords=%22data%20scientist%22&location=Germany&refresh=true
1
Page 1
<selenium.webdriver.remote.webelement.WebElement (session="09ab27f7346a5cb52217cdc7a0acdb36", element="70fe0093-aadf-4841-b0c5-653f4567ef29")>
---------------------------------------------------
------------------------------------------------------------
The current url is:https://www.linkedin.com/jobs/search/?currentJobId=3283954806&geoId=101282230&keywords=%22data%20scientist%22&location=Germany&refresh=true
2
Page 2
<selenium.webdriver.remote.webelement.WebElement (session="09ab27f7346a5cb52217cdc7a0acdb36", element="9b55fa28-5c62-4e29-ad35-e9e4548040fe")>
---------------------------------------------------
------------------------------------------------------------
The current url is:https://www.linkedin.com/jobs/search/?currentJobId=3283958387&geoId=101282230&keywords=%22data%20scientist%22&location=Germany&refres